In [1]:
from tropy.learn import fit_classifier, fit_classifier_onevsall, _inrad_eigenpair, predict_onevsall
from tropy.metrics import accuracy_multiple, veronese_feature_names, print_features_per_class
from tropy.ops import veronese
from tropy.utils import simplex_lattice_points
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

np.set_printoptions(precision=3, suppress=True)

In [2]:
lattice_points, features = None, None
def class_df(class_name, size = None):
  global lattice_points, features
  df_class = pd.read_csv(class_name, delimiter=';', dtype=float).loc[:, 'fixed acidity':'alcohol']
  features = df_class.columns.to_list()
  df_class = MinMaxScaler().fit_transform(df_class)
  df_train, df_test = train_test_split(df_class, test_size=0.2, random_state=43)
  Ctrain, Ctest = df_train.T, df_test.T

  if size is not None:
    d = Ctrain.shape[0]
    lattice_points = list(simplex_lattice_points(d, size))
    Ctrain, Ctest = veronese(lattice_points, Ctrain), veronese(lattice_points, Ctest)

  print(Ctrain.shape, Ctest.shape)
  return Ctrain, Ctest

classes = ["./data/winequality-red.csv", "./data/winequality-white.csv"]

In [3]:
size = None

In [4]:
Clist_train = []
Clist_test = []
for class_name in classes:
  train, test = class_df(class_name, size)
  Clist_train.append(train)
  Clist_test.append(test)

classes = ["Red Wine", "White Wine"]

(11, 1279) (11, 320)
(11, 3918) (11, 980)


In [5]:
x, l = _inrad_eigenpair(Clist_train, N=15)
print("Apex:", x)
print("Eigval:", l)

100%|██████████| 15/15 [00:00<00:00, 1012.07it/s]

Apex: [ 0.033  0.241 -0.111 -0.318 -0.198 -0.148  0.021 -0.298  0.401  0.05
  0.327]
Eigval: 0.11619210832332949


In [6]:
predictor, sector_indicator = fit_classifier(Clist_train, x)
indicators, apices = fit_classifier_onevsall(Clist_train)

 44%|████▍     | 22/50 [00:00<00:00, 975.66it/s]


In [7]:
accuracy = accuracy_multiple(predictor, Clist_test)
accuracy_one_vs_all = accuracy_multiple(predict_onevsall(indicators, apices, Clist_train), Clist_test)
print(f"Accuracy: {round(accuracy, 3)}")
print(f"Accuracy (one-vs-all): {round(accuracy_one_vs_all, 3)}")

Accuracy: 0.756
Accuracy (one-vs-all): 0.756


In [8]:
if size == None:
  print_features_per_class(classes, features, sector_indicator)
else:
  print_features_per_class(classes, veronese_feature_names(features, lattice_points), sector_indicator)

Dominant features for each class:
- Red Wine: free sulfur dioxide
- White Wine: fixed acidity, volatile acidity, citric acid, residual sugar, chlorides, total sulfur dioxide, density, pH, sulphates, alcohol
